## Libraries importeren

In [19]:
import torch
import tensorboard
import pandas as pd
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import train_test_split

## Data pre-processen

In [8]:
df = pd.read_csv("Data/medquad.csv")
df['question'] = df['question'].astype(str)
df['answer'] = df['answer'].astype(str)

df.head()

,question,answer,source,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma


#### Train- en test data maken

In [9]:
train_data, test_data = train_test_split(df, test_size=0.2)
print(len(train_data))
print(len(test_data))

13129
3283


#### Laad het model en de tokenizer

In [16]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

#### Data tokenizen en omzetten naar hugging face dataset

In [17]:
def tokenize_data(data):
    tokenizer.pad_token = tokenizer.eos_token
    tokenized_questions = tokenizer(data['question'], padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    tokenized_answers = tokenizer(data['answer'], padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    
    return {
        "input_ids": tokenized_questions["input_ids"].squeeze(),
        "labels": tokenized_answers["input_ids"].squeeze(),
        "attention_mask": tokenized_questions["attention_mask"].squeeze()
    }
    
train_dataset = Dataset.from_pandas(train_data)
train_dataset = train_dataset.map(tokenize_data, batched=True, remove_columns=["question", "answer"])

test_dataset = Dataset.from_pandas(test_data)
test_dataset = test_dataset.map(tokenize_data, batched=True, remove_columns=["question", "answer"])

Map:   0%|          | 0/13129 [00:00<?, ? examples/s]

Map:   0%|          | 0/3283 [00:00<?, ? examples/s]

## Model trainen (doe dit alsjeblieft een non-cuda pc niet aan)

Training is vrij zwaar, ongeveer 30 minuten met een 3080. Je kan ook de training eventueel overslaan en een opgeslagen model laden.

In [ ]:
training_args = TrainingArguments(
    output_dir="Getrainde_modellen/HouseMD_model/logs",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    fp16=True,
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    learning_rate=5e-5,
    eval_strategy="epoch",
    save_strategy="no",
    logging_dir="Getrainde_modellen/HouseMD_model/logs",
    logging_steps=10,
    report_to="tensorboard",
    seed=42
)

trainer = Trainer(
    model = model,
    args= training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    )

trainer.train()
torch.save(model.state_dict(), "Getrainde_modellen/HouseMD_model/HouseMD_Model_gewichten.pth")

Epoch,Training Loss,Validation Loss
1,3.070300,3.122479


KeyboardInterrupt: 